In [ ]:
# import package
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from time import sleep
from os import listdir
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation as FA

In [ ]:
# Second Try
def smooth_data(window, beta=0.98):
    """
    Apply smoothing using Exponentially Weighted Moving Average to a window of data.
    Args:
    - window: np.array, the sliding window data (20 time steps, n_features).
    - beta: float, smoothing factor.

    Returns:
    - np.array, smoothed data.
    """
    smoothed_window = np.zeros_like(window)
    smoothed_window[0] = window[0]  # Initialize first value with no smoothing

    for t in range(1, len(window)):
        smoothed_window[t] = beta * smoothed_window[t-1] + (1 - beta) * window[t]

    return smoothed_window

def preprocess_and_accumulate(row, minmax_dict, drop_cols, window_size=20):
    global sliding_window  # Use a global variable to store the sliding window

    # Drop unnecessary columns
    row = row.drop(drop_cols)

    # Apply min-max scaling
    scaled_row = []
    for c in row.index:
        if 's' in c:
            min_key = f'{c}min'
            max_key = f'{c}max'
            epsilon = 1e-10
            scaled_value = (row[c] - minmax_dict[min_key]) / (minmax_dict[max_key] - minmax_dict[min_key] + epsilon)
            scaled_row.append(scaled_value)

    # Update the sliding window
    if len(sliding_window) >= window_size:
        sliding_window.pop(0)  # Remove the oldest data point
    sliding_window.append(row.values)  # Add the new data point

    # Check if the window is full
    if len(sliding_window) == window_size:
        # Convert sliding window to numpy array and smooth data
        window_array = np.array(sliding_window)
        smoothed_data = smooth_data(window_array)  # Apply smoothing to the entire window
        tensor_data = torch.tensor(smoothed_data, dtype=torch.float32)  # Convert to tensor
        tensor_data = tensor_data.unsqueeze(0)  # Add a batch dimension
        tensor_data = tensor_data.to('cpu')  # Move tensor to CPU if available
        return tensor_data
    else:
        return None

drop_cols = ['os3','s1','s5','s6','s10','s16','s18','s19']
minmax_dict = {}

In [ ]:
# LSTM building
class LSTMRegressor(nn.Module):

    def __init__(self):
        super().__init__()
        self.n_features = 16
        self.hidden_units = 12
        self.n_layers = 1
        self.lstm = nn.LSTM(input_size = self.n_features, hidden_size = self.hidden_units, batch_first = True, num_layers = self.n_layers)
        self.linear1 = nn.Linear(in_features=self.hidden_units, out_features=12)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(in_features=12, out_features=12)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(in_features=12, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear1(hn[0])
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        out = self.linear3(out).flatten()

        return out

In [ ]:
model = LSTMRegressor()

In [ ]:
model.load_state_dict(torch.load("model.pth"))
model.eval()

LSTMRegressor(
  (lstm): LSTM(16, 12, batch_first=True)
  (linear1): Linear(in_features=12, out_features=12, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=12, out_features=12, bias=True)
  (relu2): ReLU()
  (linear3): Linear(in_features=12, out_features=1, bias=True)
)

In [ ]:
col_names = ['unit', 'time', 'os1', 'os2', 'os3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [ ]:
len(col_names)

26

In [ ]:
len(drop_cols)

8

In [ ]:
#df_train = pd.read_csv((dir_path+'train_FD001.txt'), sep=" ", header=None, names=col_names)
df_test = pd.read_csv('test_FD001.txt', sep=" ", header=None, names=col_names)
#rul_test = pd.read_csv((dir_path+'RUL_FD001.txt'), sep=" ", header=None, names=['RUL'])

In [ ]:
df_test

unit    time    os1     os2     os3       s1       s2     s3  \
1   1    0.0023  0.0003  100.0  518.67  643.02  1585.29  1398.21  14.62   
    2   -0.0027 -0.0003  100.0  518.67  641.71  1588.45  1395.42  14.62   
    3    0.0003  0.0001  100.0  518.67  642.46  1586.94  1401.34  14.62   
    4    0.0042  0.0000  100.0  518.67  642.44  1584.12  1406.42  14.62   
    5    0.0014  0.0000  100.0  518.67  642.51  1587.19  1401.92  14.62   
...         ...     ...    ...     ...     ...      ...      ...    ...   
100 194  0.0049  0.0000  100.0  518.67  643.24  1599.45  1415.79  14.62   
    195 -0.0011 -0.0001  100.0  518.67  643.22  1595.69  1422.05  14.62   
    196 -0.0006 -0.0003  100.0  518.67  643.44  1593.15  1406.82  14.62   
    197 -0.0038  0.0001  100.0  518.67  643.26  1594.99  1419.36  14.62   
    198  0.0013  0.0003  100.0  518.67  642.95  1601.62  1424.99  14.62   

            s4      s5  ...      s12     s13   s14  s15   s16    s17    s18  \
1   1    21.61  553.90  ...  8125.55  8.4052  0.03  392  2388  100.0  38.86   
    2    21.61  554.85  ...  8139.62  8.3803  0.03  393  2388  100.0  39.02   
    3    21.61  554.11  ...  8130.10  8.4441  0.03  393  2388  100.0  39.08   
    4    21.61  554.07  ...  8132.90  8.3917  0.03  391  2388  100.0  39.00   
    5    21.61  554.16  ...  8129.54  8.4031  0.03  390  2388  100.0  38.99   
...        ...     ...  ...      ...     ...   ...  ...   ...    ...    ...   
100 194  21.61  553.41  ...  8213.28  8.4715  0.03  394  2388  100.0  38.65   
    195  21.61  553.22  ...  8210.85  8.4512  0.03  395  2388  100.0  38.57   
    196  21.61  553.04  ...  8217.24  8.4569  0.03  395  2388  100.0  38.62   
    197  21.61  553.37  ...  8220.48  8.4711  0.03  395  2388  100.0  38.66   
    198  21.61  552.48  ...  8214.64  8.4903  0.03  396  2388  100.0  38.70   

             s19  s20  s21  
1   1    23.3735  NaN  NaN  
    2    23.3916  NaN  NaN  
    3    23.4166  NaN  NaN  
    4    23.3737  NaN  NaN  
    5    23.4130  NaN  NaN  
...          ...  ...  ...  
100 194  23.1974  NaN  NaN  
    195  23.2771  NaN  NaN  
    196  23.2051  NaN  NaN  
    197  23.2699  NaN  NaN  
    198  23.1855  NaN  NaN  

[13096 rows x 26 columns]

In [ ]:
for c in df_test.columns:
    if 's' in c:
        minmax_dict[c+'min'] = df_test[c].min()
        minmax_dict[c+'max']=  df_test[c].max()

# Initialize the sliding window
sliding_window = []

In [ ]:
df_test.drop(['unit','time'],axis=1,inplace=True)

In [ ]:
def replace_nans(tensor):
    if tensor is None:
        return None
    return torch.nan_to_num(tensor, nan=0.0)

In [ ]:
# Example of how you might call this in your prediction loop

# Model Loading and Evaluation: Make sure that model is correctly loaded and set to evaluation mode (model.eval()) before making predictions.
for index, row in df_test.sample(30).iterrows():
    #tensor_data = preprocess_and_accumulate(row, minmax_dict, drop_cols)
    tensor_data = replace_nans(preprocess_and_accumulate(row, minmax_dict, drop_cols))
    #print(tensor_data)
    if tensor_data is not None:
        prediction = model(tensor_data)  # Ensure model is properly defined and loaded
        print(prediction)
        # Handle the prediction as needed

tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)
tensor([211.3666], grad_fn=<ViewBackward0>)


In [ ]:
tensor_data = preprocess_and_accumulate(row, minmax_dict, drop_cols)
if torch.isnan(tensor_data).any():
    print("NaN detected in input data:", tensor_data)

NaN detected in input data: tensor([[[1.0000e+02, 5.1867e+02, 1.4083e+03, 1.4620e+01, 2.1610e+01,
          9.0512e+03, 1.3000e+00, 4.7470e+01, 2.3881e+03, 8.1268e+03,
          8.4420e+00, 3.0000e-02, 3.9300e+02, 1.0000e+02,        nan,
                 nan],
         [1.0000e+02, 5.1867e+02, 1.4084e+03, 1.4620e+01, 2.1610e+01,
          9.0513e+03, 1.3000e+00, 4.7470e+01, 2.3881e+03, 8.1271e+03,
          8.4422e+00, 3.0000e-02, 3.9304e+02, 1.0000e+02,        nan,
                 nan],
         [1.0000e+02, 5.1867e+02, 1.4085e+03, 1.4620e+01, 2.1610e+01,
          9.0520e+03, 1.3000e+00, 4.7471e+01, 2.3881e+03, 8.1277e+03,
          8.4417e+00, 3.0000e-02, 3.9304e+02, 1.0000e+02,        nan,
                 nan],
         [1.0000e+02, 5.1867e+02, 1.4084e+03, 1.4620e+01, 2.1610e+01,
          9.0520e+03, 1.3000e+00, 4.7470e+01, 2.3881e+03, 8.1279e+03,
          8.4410e+00, 3.0000e-02, 3.9304e+02, 1.0000e+02,        nan,
                 nan],
         [1.0000e+02, 5.1867e+02, 1.4084